# MongoDB简介
[官网](https://www.mongodb.com/) | [文档入口](https://docs.mongodb.com/) | [MongoDB Python Drivers文档](https://docs.mongodb.com/ecosystem/drivers/python/) | [MongoDB 大学](https://university.mongodb.com/courses/M001/about)

## 安装

1. 下载[MongoDB Server](https://www.runoob.com/mongodb/mongodb-osx-install.html) | [PyMongo](https://www.runoob.com/python3/python-mongodb.html)
* 先创建数据存储地址 `mkdir -p ./data/db`
* 再启动 mongo 服务端 `mongod --dbpath=./data/db`
* 最后`import pymongo` 就可以操作了

## 概念
数据结构
* Databases 数据库
* Collections 表
* Documents JSON格式的数据

数据操作
* CRUD (Create Read Update Delete) 增删改查 

工具
* MongoDB Atlas 云服务
* MongoDB Compass 数据管理客户端

## 数据操作
[PyMongo文档](https://pymongo.readthedocs.io/en/stable/)

In [ ]:
from pymongo import MongoClient,DESCENDING
client = MongoClient('mongodb://127.0.0.1:27017')

### 新增

In [ ]:
db = client.test_db
users = db.users
result = users.insert_one({'name':'hawk'})

In [ ]:
result.inserted_id

### 查询

In [ ]:
client.list_database_names()

In [ ]:
db.list_collection_names()

In [ ]:
users.find_one()

In [ ]:
for item in users.find({'name': 'hawk'}):
    print(item)

#### 计数

In [ ]:
users.count_documents({'name': 'hawk'})

#### 排序

In [ ]:
for item in users.find().sort('_id', DESCENDING):
    print(item)

#### 其他

In [ ]:
for item in users.find().sort('_id', DESCENDING).skip(1).limit(2):
    print(item)

### 删除

In [ ]:
users.delete_one({'name':'hawk'})

In [ ]:
db.drop_collection('users')

In [ ]:
client.drop_database('test_db')

### 修改
[Update Operator文档](https://docs.mongodb.com/manual/reference/operator/update/)

In [ ]:
users.update_one({'name':'hawk'},{'$set':{'age':30}})
users.update_one({'age':30},{'$inc':{'age':2}})

In [ ]:
for item in users.find():
    print(item)

### 筛选条件

In [ ]:
%%html
<table><thead><tr><th>符号</th><th>含义</th><th>示例</th></tr></thead><tbody><tr><td><code>$lt</code></td><td>小于</td><td><code>{'age': {'$lt': 20}}</code></td></tr><tr><td><code>$gt</code></td><td>大于</td><td><code>{'age': {'$gt': 20}}</code></td></tr><tr><td><code>$lte</code></td><td>小于等于</td><td><code>{'age': {'$lte': 20}}</code></td></tr><tr><td><code>$gte</code></td><td>大于等于</td><td><code>{'age': {'$gte': 20}}</code></td></tr><tr><td><code>$ne</code></td><td>不等于</td><td><code>{'age': {'$ne': 20}}</code></td></tr><tr><td><code>$in</code></td><td>在范围内</td><td><code>{'age': {'$in': [20, 23]}}</code></td></tr><tr><td><code>$nin</code></td><td>不在范围内</td><td><code>{'age': {'$nin': [20, 23]}}</code></td></tr></tbody></table>

In [ ]:
%%html
<table><thead><tr><th>符号</th><th>含义</th><th>示例</th><th>示例含义</th></tr></thead><tbody><tr><td><code>$regex</code></td><td>匹配正则表达式</td><td><code>{'name': {'$regex': '^M.*'}}</code></td><td><code>name</code>以M开头</td></tr><tr><td><code>$exists</code></td><td>属性是否存在</td><td><code>{'name': {'$exists': True}}</code></td><td><code>name</code>属性存在</td></tr><tr><td><code>$type</code></td><td>类型判断</td><td><code>{'age': {'$type': 'int'}}</code></td><td><code>age</code>的类型为<code>int</code></td></tr><tr><td><code>$mod</code></td><td>数字模操作</td><td><code>{'age': {'$mod': [5, 0]}}</code></td><td>年龄模5余0</td></tr><tr><td><code>$text</code></td><td>文本查询</td><td><code>{'$text': {'$search': 'Mike'}}</code></td><td><code>text</code>类型的属性中包含<code>Mike</code>字符串</td></tr><tr><td><code>$where</code></td><td>高级条件查询</td><td><code>{'$where': 'obj.fans_count == obj.follows_count'}</code></td><td>自身粉丝数等于关注数</td></tr></tbody></table>

详细规则 https://docs.mongodb.com/manual/reference/operator/query/

### 合并操作

In [ ]:
# find_one_and_delete
# find_one_and_replace
users.find_one_and_update({'age':32},{'$inc':{'age':2}})

# 实践案例，爬人人都是产品经理网站
> MongoDB的存储位置为./data/db

现存问题：
- [x] 爬取url太慢：用代理并行去爬
- [ ] 爬url，经常overtry，因为网站经常会卡死，设定workers少一点

In [ ]:
# default_exp RRPM

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# export
import requests,re,time,sys
from bs4 import BeautifulSoup
from crawler_from_scratch import IpPool
from pymongo import MongoClient
from datetime import datetime


In [ ]:
# export
client = MongoClient('mongodb://127.0.0.1:27017')
db = client.crawler
art_coll = db.articals

category = 'pd'

In [ ]:
# export
def simple_request(url) -> object:
    res = requests.get(url,headers={'user-agent':'Mozilla/5.0'})
    soup = BeautifulSoup(res.text)
    return soup.body

## 生成待爬url list


In [ ]:
#export
def crawl_artical_url_v1(category_page_url) -> iter:
    res = requests.get(category_page_url)
    soup = BeautifulSoup(res.text,features='lxml').body
    for s in soup.find_all('h2','post-title'):
        url = s.a['href']
        if 'woshipm' in url: yield url
    
def get_urls_v1(start,end) -> list:
    global category
    urls = []
    category_url = f'http://www.woshipm.com/category/{category}'
    for i in range(start,end+1):
        page_url = f'{category_url}/page/{i}'
        print(page_url)
        for url in crawl_artical_url_v1(page_url):
            urls.append(url)
    print('urls:',len(urls),'per_page:',len(urls)/(end-start+1))
    urls = list(set(urls))
    return urls

In [ ]:
get_urls_v1(20,30)

In [ ]:
#export
def crawl_artical_url(category_page_url) -> list:
    urls = []
    res = IpPool.proxy_get(category_page_url)
    soup = BeautifulSoup(res.text,features='lxml').body
    for s in soup.find_all('h2','post-title'):
        url = s.a['href']
        if 'woshipm' in url: urls.append(url)
    
    # 获取失败的情况
    if len(urls) == 0: 
        print('\n\n!no urls:',category_page_url, 'retrying......')
        urls = crawl_artical_url(category_page_url)
    print('crawl urls:',category_page_url,'got:',len(urls))
    return urls

def parallel_crawl_artical_url(start,end) -> list:
    global category
    urls = []
    page_urls = [f'http://www.woshipm.com/category/{category}/page/{i}' for i in range(start,end+1)]
    for data in IpPool.parallel_task(crawl_artical_url,page_urls,10):
        urls += data
    urls = list(set(urls))
    print('urls:',len(urls))
    return urls

In [ ]:
crawl_artical_url('http://www.woshipm.com/category/it/page/2')

https://www.7yip.cn/free/ 新增：0，库存更新为：2122个
http://www.ip3366.net/free/ 新增：4，库存更新为：2126个
http://www.nimadaili.com/gaoni/ 新增：37，库存更新为：2163个
http://www.xiladaili.com/gaoni/ 新增：3，库存更新为：2166个
https://www.kuaidaili.com/free/inha/ 新增：0，库存更新为：2167个
https://ip.jiangxianli.com/?anonymity=2 新增：7，库存更新为：2173个
http://proxyslist.com/ 新增：23，库存更新为：2196个
https://list.proxylistplus.com/Fresh-HTTP-Proxy-List-1 新增：1，库存更新为：2197个
移除： 0 个IP
success: http://www.woshipm.com/category/it/page/2 try times: 1
crawl urls: http://www.woshipm.com/category/it/page/2 got: 12


In [ ]:
category = 'it'
urls = parallel_crawl_artical_url(1,100)

In [ ]:
urls

['http://www.woshipm.com/it/3690544.html',
 'http://www.woshipm.com/it/3690303.html',
 'http://www.woshipm.com/it/3679467.html',
 'http://www.woshipm.com/it/3689204.html',
 'http://www.woshipm.com/it/3688145.html',
 'http://www.woshipm.com/it/3658393.html',
 'http://www.woshipm.com/it/3685361.html',
 'http://www.woshipm.com/it/3685687.html',
 'http://www.woshipm.com/it/3684275.html',
 'http://www.woshipm.com/it/3679947.html',
 'http://www.woshipm.com/it/3684034.html',
 'http://www.woshipm.com/it/3678488.html',
 'http://www.woshipm.com/it/3678488.html',
 'http://www.woshipm.com/it/3686449.html',
 'http://www.woshipm.com/it/3686307.html',
 'http://www.woshipm.com/it/3683307.html',
 'http://www.woshipm.com/it/3679936.html',
 'http://www.woshipm.com/it/3684661.html',
 'http://www.woshipm.com/it/3682891.html',
 'http://www.woshipm.com/it/3678595.html',
 'http://www.woshipm.com/it/3678962.html',
 'http://www.woshipm.com/it/3677359.html',
 'http://www.woshipm.com/it/3677782.html',
 'http://ww

## 代理访问&保存html

In [ ]:
# export
def save_artical(response) -> int:
    global category
    url = response.url
    _id = int(re.search(r'\d+',url).group())
    
    soup = BeautifulSoup(response.text,features='lxml')
    content = str(soup.find('div','single-wrapper'))

    try:
        artical = art_coll.insert_one({'_id':_id,
                                       'category':category,
                                       'url':url,
                                       'html':content})
        return artical.inserted_id
    except:
        print('\n\n!error',url,sys.exc_info())
    
def crawl_artical(url):
    global category,art_coll
    _id = int(re.search(r'\d+',url).group())
    
    # 爬过，则结束
    if art_coll.find_one({'_id':_id}):
        print('exist:',url)
        return
    
    # 重复爬10次，直到res=200，不然就log
    res = IpPool.proxy_get(url) 
    if res: save_artical(res)          

In [ ]:
crawl_artical('http://www.woshipm.com/rp/3660512.html')

exist: http://www.woshipm.com/rp/3660512.html


## 批量操作

In [ ]:
# export
def crawl_artical_url_and_html(category_page_url):
    urls = crawl_artical_url(category_page_url)
    IpPool.parallel_task(crawl_artical,urls,10,False)
    print('complete:',category_page_url)
    return True

In [ ]:
# export
def crawl_artical_by_category():
    global category
    category = 'blockchain'
    start,end = 1,20
    category_page_urls = [f'http://www.woshipm.com/category/{category}/page/{i}' for i in range(start,end+1)]
    IpPool.parallel_task(crawl_artical_url_and_html,category_page_urls,20)
#     urls = parallel_crawl_artical_url(1,133)    
# #     urls = get_urls_v1(1,257)
#     IpPool.parallel_task(crawl_artical,urls)

## 完善数据

In [ ]:
def check_no_html() -> list:
    no_html_urls = []
#     for a in art_coll.find({'html':{'$exists':True},'$where':"(this.html.length < 100)"}):
    for a in art_coll.find({'html':{'$exists':False}}):
        no_html_urls.append(a['url'])
    print(len(no_html_urls))
    return no_html_urls

In [ ]:
def update_html(url):
    res = IpPool.proxy_get(url)
    soup = BeautifulSoup(res.text,features='lxml')
    content = str(soup.find('div','single-wrapper'))
    result = art_coll.update_one({'url':url},{'$set':{'html':content}})
    print('update:',url,result.modified_count)

In [ ]:
check_no_html()

0


[]

In [ ]:
IpPool.parallel_task(update_html,check_no_html())

## 解析内容字段

In [ ]:
# export
def parse_ymd(s):
    year_s, mon_s, day_s = s.split('-')
    return datetime(int(year_s), int(mon_s), int(day_s))

In [ ]:
# export
def parse_html(_id=3614430) -> object:
    update_content = {}
    try:
        soup = BeautifulSoup(art_coll.find_one({'_id':_id})['html'],features='lxml')
        update_content['title'] = soup.h2.text
        author_soup = soup.find('div','postMetaLockup--authorWithBio u-flex')
        update_content['author_link'] = author_soup.a['href']

        author_info = [s for s in author_soup.stripped_strings]
        update_content['author_name'] = author_info[0]
        update_content['likes'] = int(author_info[-1])
        update_content['stars'] = int(author_info[-2])
        update_content['views'] = int(author_info[-3])
        update_content['publish_date'] = parse_ymd(author_info[-4])
        update_content['tags'] = [a.text for a in soup.find('div','taglist')] if soup.find('div','taglist') else []
    except:
        print('parse error:',_id)
    return update_content

In [ ]:
# export
def update_artical_info(_id) -> int: 
    result = art_coll.update_one({'_id':_id},{'$set':parse_html(_id)})
    return result.modified_count

In [ ]:
parse_html(1633891)

parse error: 1633891


{'title': '如何报名'}

In [ ]:
update_artical_info(3614430)

0

In [ ]:
# export
# http://www.woshipm.com/it/793639.html
# http://www.woshipm.com/it/1633891.html
# 这两个是培训课程，需删掉
def update_all_artical_info():
    ids = [i['_id'] for i in art_coll.find({'likes':{'$exists':False}})]
    task_results = IpPool.parallel_task(update_artical_info,ids,1)
    modify_count = len([i for i in task_results if i> 0])
    print('modify:',modify_count)

## 检查数据完整性

In [ ]:
# export
def correct_field_type(_id=3614430):
    art = art_coll.find_one({'_id':_id})
    if type(art['views']) == str:
        if '万' in art['views']:
            art['views'] = float(art['views'].replace('万',''))*10000
        elif 'm' in art['views']:
            art['views'] = float(art['views'].replace('m',''))*1000000
            
        result = art_coll.update_one({'_id':_id},
                                     {'$set':{'stars':int(art['stars']),
                                              'likes':int(art['likes']),
                                              'views':int(art['views']),
                                              'publish_date':parse_ymd(art['publish_date'])}})
        return result.modified_count
    

In [ ]:
correct_field_type(3624725)

In [ ]:
!nbdev_build_lib --fname 24_MongoDB.ipynb
!mv crawler_from_scratch/RRPM.py RRPM.py 

Converted 24_MongoDB.ipynb.


In [ ]:
# export
if __name__ == '__main__': 
    ids = [i['_id'] for i in art_coll.find()]
    IpPool.parallel_task(correct_field_type,ids,10)

progress:20.00%
time cost: 0 s 5 tasks


## 数据库导出备份
https://cloud.tencent.com/document/product/240/5321

### 导出

In [ ]:
test_coll = db.test
test_coll.insert_many(art_coll.find().limit(10))

In [ ]:
# mongoexport --host 10.66.187.127:27017 -u mongouser -p thepasswordA1 --authenticationDatabase=admin --db=testdb --collection=testcollection -o /data/export_testdb_testcollection.json
!mongoexport --host 127.0.0.1:27017 --db=crawler --collection=test -o ./data/export_test_coll.json
    
    

2020-04-17T09:17:11.211+0800	connected to: mongodb://127.0.0.1:27017/
2020-04-17T09:17:11.227+0800	exported 10 records


### 导入
可以识别date格式

In [ ]:
# mongoimport --host 10.66.187.127:27017 -u mongouser -p thepasswordA1 --authenticationDatabase=admin --db=testdb --collection=testcollection2 --file=/data/export_testdb_testcollection.json
!mongoimport --host 127.0.0.1:27017 --db=crawler --collection=test2 --file=./data/export_test_coll.json
        
        

2020-04-17T09:19:12.501+0800	connected to: mongodb://127.0.0.1:27017/
2020-04-17T09:19:12.573+0800	10 document(s) imported successfully. 0 document(s) failed to import.


### rrpm文章数据备份

In [ ]:
# 所有文章的html源文件
rrpm_raw_coll = db.rrpm_raw
for i in art_coll.find():
# for i in art_coll.find().limit(5):
    rrpm_raw_coll.insert_one({'html':i['html'],'url':i['url']})

In [ ]:
!mongoexport --host 127.0.0.1:27017 --db=crawler --collection=rrpm_raw -o ./data/export_rrpm_raw.json


### 重新建立字段数据库
把有用的都保存下来，删除原始数据

In [ ]:
# export
def parse_ymd(s):
    year_s, mon_s, day_s = s.split('-')
    return datetime(int(year_s), int(mon_s), int(day_s))

def parse_num(string):
    if '万' in string:
        return int(float(string.replace('万',''))*10000)
    elif 'm' in string:
        return int(float(string.replace('m',''))*1000000)
    else:
        try:
            return int(string)
        except:
            print('error:',string)

In [ ]:
def parse_comment(soup) -> object:
    cmt = {}
    comment = soup
    comment_author = comment.find('div','comment-author')
    if comment_author.a:
        cmt['author_name'] = comment_author.a.text.strip()
        cmt['author_link'] = comment_author.a['href']
    cmt['content'] = comment.find('div','comment-content').text.strip()
    cmt['comment_time'] = datetime.strptime(comment.find('span','comment-time')['datetime'],"%Y-%m-%dT%H:%M:%S+08:00")
    return cmt

def parse_comments(soup) -> list:
    cmt_list = []
    for item in soup.find_all('li','comment'): 
        cmt_list.append(parse_comment(item))
    return cmt_list

In [ ]:
def parse_artical(soup) -> object:
    artical = {}
    
    artical['title'] = soup.h2.text
    artical['publish_date'],artical['views'],artical['stars'],artical['likes'] = [item.text.strip() for item in soup.find_all('span','post-meta-item')]
    
    author_info = soup.find('div','postMetaLockup--authorWithBio')    
    artical['author_name'] = author_info.find_all('a')[1].text.strip()    
    artical['author_link'] = author_info.find_all('a')[1]['href']    
    artical['author_desc'] = author_info.find('div','des').text.strip()  
    
    main = soup.find('div','grap')
    artical['contents'] = list(main.stripped_strings)
    artical['imgs'] = [item['src'] for item in main.find_all('img') if 'src' in item.attrs]
    
    artical['tags'] = [item.text for item in soup.find('div','taglist')] if soup.find('div','taglist') else []
    
    artical['publish_date'] = parse_ymd(artical['publish_date'])
    artical['views'] = parse_num(artical['views'])
    artical['stars'] = parse_num(artical['stars'])
    artical['likes'] = parse_num(artical['likes'])
    
    
    artical['comments'] = parse_comments(soup)
    return artical

In [ ]:
def save_artical_to_mongodb(raw_item):
    try:
        soup = BeautifulSoup(raw_item['html'],features='lxml')
        _id = int(re.search(r'\d+',raw_item['url']).group())
        artical = parse_artical(soup)
        artical['_id'] = _id
        artical['url'] = raw_item['url']
        if not rrpm_articals_coll.insert_one(artical):
            print('save failed:',raw_item['url'])
            with open('failed_urls.text','a') as f:
                f.write(raw_item['url']+'\n')

    except:
        print('parse error:',raw_item['url'])
        with open('failed_urls.text','a') as f:
            f.write(raw_item['url']+'\n')
    

In [ ]:
rrpm_articals_coll = db.rrpm_articals

In [ ]:
items = rrpm_raw_coll.find()
IpPool.parallel_task(save_artical_to_mongodb,items)

## 发布

In [ ]:
!git add 24_MongoDB.ipynb
!git commit -m "restruct database"

[master b5eeff2] restruct database
 1 file changed, 491 insertions(+), 10 deletions(-)


In [ ]:
!nbdev_build_lib --fname 24_MongoDB.ipynb
!mv crawler_from_scratch/RRPM.py RRPM.py 